In [7]:
from google.colab import drive
drive.mount( '/content/drive' )
directory = ("/content/drive/MyDrive/Beca CONICET/Desarrollo/Liquid Crystals Micro/Programas/LAMMPS/2023/Barrido largo y flexibilidad/")
import os
os.chdir(directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%%capture
!pip install -U ovito

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
from DOSPT import *
from scipy.constants import Boltzmann #light speed in [m/s], #Planck's constant  [J*s]

#-----Simulation type and parameters------
variable = 'P'

#DOS type
stat = 'q'

#Adim param
t = 1.0

#CH2 parameters (From McCoy1995)
Sigma = 3.56 * 1.e-10                             #Sigma in [m]
kb = Boltzmann                                      #Boltzmann's constant [J/K]
Epsilon = 70.47 * kb                              #Epsilon in [J]
mch2 = 14.026 *(1. / 1000.)/6.02214076e23        #CH2 mass in [kg]

#Simulation type and time
nruns = 20
ens = 'NVT'
Dstep = 16               #frames dump
tstep = 0.001*np.sqrt(mch2 / Epsilon)*Sigma  #time step in second
skip = 550                #If don't want load all frames, can jump frames
fcv = (Sigma / (np.sqrt(mch2 / Epsilon)*Sigma))    #conversion factor lj units to [m/s]
fcx = Sigma                                      #conversion factor lj to SI in [m]
SN = 2.                                        #Symmetry Number: Molecula lineal homonuclear = 2 (Ver:TEMA 1. Termodinámica Estadística)
T   =  t * (Epsilon/kb)


l0lps = [0.05, 0.275, 0.5]
nbs = [13, 15, 17]

#Pressures
ps = {}  # Crea un diccionario vacío

for l0lp in l0lps:
    for nb in nbs:
        ps[(l0lps, nbs)] : None  # Agrega los valores de "p"

# Completar el diccionario con los valores de "p" que tengas [pISO, pNm]
ps[(0.05, 13)] = [0.8, 1.0]
ps[(0.05, 15)] = [0.4, 0.6]
ps[(0.05, 17)] = [0.4, 0.5]
ps[(0.275, 13)] = [1.6, 1.8]
ps[(0.275, 15)] = [1.0, 1.2]
ps[(0.275, 17)] = [0.65, 0.75]
ps[(0.5, 13)] = [2.6, 2.8]
ps[(0.5, 15)] = [1.5, 1.8]
ps[(0.5, 17)] = [1.1, 1.2]

for l0lp in l0lps:
    for nb in nbs:
        m = np.tile(mch2, nb)  #Atoms mass in kg
        for p in ps[(l0lp, nb)]:
            folder = variable + (str(t) if variable == 'T' else str(p)) if variable in ['T', 'P'] else None
            root =  (f'l0lp{l0lp}/nb{nb}/{variable}_variable/{folder}/')
            os.makedirs(root+'PostProc/DOS_thermo/', exist_ok=True)
            for dof in ['trn', 'rot', 'vib']:
                thermo  = np.zeros((nruns,4))
                for i in np.arange(1,nruns+1):
                    V  = np.genfromtxt(root + f'thermo_nvt_{i}_t{t}p{p}.log' , skip_header=45,
                                    invalid_raise = False)[0,5]*(Sigma**3) #vol in [m3]
                    rho =  (16*16*18*nb*mch2)/V         #Density in [Kg/m3]

                    #INPUT
                    file_in =  root + (f'PostProc/Dos{dof}{ens}_{i}.DS{Dstep}t*{t}p*{p}.txt')
                    file_in_trj =  root + (f'PostProc/Dos{dof}{ens}_{i}.DS{Dstep}t*{t}p*{p}.trj')
                    print('Procesando:',file_in)
                    coords = Trajectory(file_in_trj, 0,skip,fcv,fcx) #solo para tener el n de atomos
                    DOS_data = np.genfromtxt(file_in, skip_header=0,invalid_raise = False)[:,:]
                    DOSt = np.zeros((nruns,4))
                    freq, DOSt= DOS_data[:,0], DOS_data[:,1:]
                    if dof == 'trn':
                        cDOSt, cDOSg, cDOSs, vfreq, f, Delta= coords.Return_DOS_partition_trn(DOSt[:,0],freq,tstep,Dstep,T,m,nb,rho)
                        Ei, Si, Ai = coords.Return_thermoproperties_trn(cDOSg,cDOSs,vfreq,T,m,f,Delta,V,nb,stat)
                        thermo[i-1,:] = [f,Ei,Si,Ai]

                    elif dof == 'rot':
                        with open(dir) as f:
                            string = f.readline().replace(']','').replace('[','').split()[-3:]
                            I = [float(string[0]), float(string[1]), float(string[2])]

                        cDOSt, cDOSg, cDOSs, vfreq, f, Delta= coords.Return_DOS_partition_rot(DOSt[:,0],freq,tstep,Dstep,T,m,nb,rho)
                        Ei, Si, Ai = coords.Return_thermoproperties(cDOSg,cDOSs,vfreq,T,I,f,Delta,V,nb,stat,sigma=2.)
                        thermo[i-1,:] = [f,Ei,Si,Ai]

                    elif dof == 'vib':
                        cDOSt, cDOSs, vfreq= coords.Return_DOS_partition_vib(DOSt[:,0],freq)
                        Ei, Si, Ai = coords.Return_thermoproperties(cDOSs,vfreq,T,nb,stat)
                        f = 0.
                        thermo[i-1,:] = [f,Ei,Si,Ai]

                #OUTPUT
                file_out =  root + (f'PostProc/DOS_thermo/DOS_thermo_{dof}{ens}.DS{Dstep}t*{t}p*{p}.txt')
                out_file = open(file_out, "w")
                np.savetxt(out_file, thermo[:,:],\
                            fmt='%E',header='f [], E/p [J], S/p [J/K], A/p [J]')
                out_file.close()
                print('Procesado:', file_in)

